In [13]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import json
from pyproj import Geod
from typing import List, Tuple, Dict, Any
import pandas as pd
import folium


In [14]:
# Define OSM tags for waypoints
WATER_TAGS = {
    'amenity': ['drinking_water']
}

STORE_TAGS = {
    'shop': ['convenience', 'supermarket', 'grocery', 'beverages']
}

# Philadelphia bounding box (same as in build_graph_2.py)
PHILLY_BBOX = (40.0, 39.9, -75.1, -75.2)  # (north, south, east, west)

print("Fetching water fountains and stores from OpenStreetMap...")


Fetching water fountains and stores from OpenStreetMap...


In [15]:
# Fetch water fountains
print("Fetching water fountains...")
north, south, east, west = PHILLY_BBOX
# Use bbox parameter instead of individual north/south/east/west
water_gdf = ox.features_from_bbox(bbox=(west, south, east, north), tags=WATER_TAGS)
print(f"Found {len(water_gdf)} water fountains")

# Fetch stores
print("Fetching stores...")
store_gdf = ox.features_from_bbox(bbox=(west, south, east, north), tags=STORE_TAGS)
print(f"Found {len(store_gdf)} stores")

# Check what columns are available
print(f"\nWater fountains columns: {list(water_gdf.columns)}")
print(f"Stores columns: {list(store_gdf.columns)}")

# Display sample data (handle missing columns gracefully)
if len(water_gdf) > 0:
    print("\nSample water fountain:")
    available_cols = [col for col in ['amenity', 'name', 'geometry'] if col in water_gdf.columns]
    if available_cols:
        print(water_gdf[available_cols].head(1))
    else:
        print("No expected columns found")

if len(store_gdf) > 0:
    print("\nSample store:")
    available_cols = [col for col in ['shop', 'name', 'geometry'] if col in store_gdf.columns]
    if available_cols:
        print(store_gdf[available_cols].head(1))
    else:
        print("No expected columns found")


Fetching water fountains...
Found 9 water fountains
Fetching stores...
Found 325 stores

Water fountains columns: ['geometry', 'amenity', 'tourism', 'check_date', 'bottle', 'fountain', 'access', 'fee', 'wheelchair', 'dog']
Stores columns: ['geometry', 'addr:city', 'addr:housenumber', 'addr:postcode', 'addr:state', 'addr:street', 'branch', 'brand', 'brand:wikidata', 'check_date', 'check_date:opening_hours', 'name', 'opening_hours', 'phone', 'ref', 'shop', 'source', 'website', 'wheelchair', 'addr:housename', 'vacant', 'atm', 'delivery', 'fast_food', 'fuel:propane', 'cuisine', 'name:zh', 'wheelchair:description', 'contact:website', 'operator:wikidata', 'payment:app', 'payment:apple_pay', 'payment:cards', 'payment:cash', 'payment:contactless', 'payment:credit_cards', 'second_hand', 'opening_hours:signed', 'contact:phone', 'opening_hours:covid19', 'name:en', 'building', 'air_conditioning', 'changing_table', 'payment:debit_cards', 'toilets', 'drive_through', 'outdoor_seating', 'takeaway', 'i

In [16]:
def extract_waypoint_data(gdf, waypoint_type: str) -> List[Dict[str, Any]]:
    """Extract waypoint data from GeoDataFrame into structured format"""
    waypoints = []
    for idx, row in gdf.iterrows():
        geom = row.geometry
        if geom and not geom.is_empty:
            if geom.geom_type == 'Point':
                lon, lat = geom.x, geom.y
                # Handle missing name column gracefully
                name = row.get('name', None) or row.get('amenity', None) or row.get('shop', None) or f'{waypoint_type}_{idx}'
                waypoint_data = {
                    'id': f"{waypoint_type}_{idx}",
                    'type': waypoint_type,
                    'name': str(name),
                    'coordinates': [lat, lon],  # [lat, lng] for frontend compatibility
                    'longitude': lon,
                    'latitude': lat,
                    'amenity': str(row.get('amenity', '')),
                    'shop': str(row.get('shop', '')),
                    'opening_hours': str(row.get('opening_hours', '')),
                    'website': str(row.get('website', '')),
                    'phone': str(row.get('phone', ''))
                }
                waypoints.append(waypoint_data)
            elif geom.geom_type == 'MultiPoint':
                for i, point in enumerate(geom.geoms):
                    lon, lat = point.x, point.y
                    name = row.get('name', None) or row.get('amenity', None) or row.get('shop', None) or f'{waypoint_type}_{idx}_{i}'
                    waypoint_data = {
                        'id': f"{waypoint_type}_{idx}_{i}",
                        'type': waypoint_type,
                        'name': str(name),
                        'coordinates': [lat, lon],
                        'longitude': lon,
                        'latitude': lat,
                        'amenity': str(row.get('amenity', '')),
                        'shop': str(row.get('shop', '')),
                        'opening_hours': str(row.get('opening_hours', '')),
                        'website': str(row.get('website', '')),
                        'phone': str(row.get('phone', ''))
                    }
                    waypoints.append(waypoint_data)
    return waypoints

# Extract waypoints
water_waypoints = extract_waypoint_data(water_gdf, 'water')
store_waypoints = extract_waypoint_data(store_gdf, 'store')

print(f"Extracted {len(water_waypoints)} water waypoints")
print(f"Extracted {len(store_waypoints)} store waypoints")

# Sample waypoints
if water_waypoints:
    print(f"\nSample water waypoint: {water_waypoints[0]}")
if store_waypoints:
    print(f"Sample store waypoint: {store_waypoints[0]}")


Extracted 9 water waypoints
Extracted 198 store waypoints

Sample water waypoint: {'id': "water_('node', 1586414753)", 'type': 'water', 'name': 'drinking_water', 'coordinates': [39.9662606, -75.1826961], 'longitude': -75.1826961, 'latitude': 39.9662606, 'amenity': 'drinking_water', 'shop': '', 'opening_hours': '', 'website': '', 'phone': ''}
Sample store waypoint: {'id': "store_('node', 333786275)", 'type': 'store', 'name': 'Whole Foods Market', 'coordinates': [39.9428324, -75.1581249], 'longitude': -75.1581249, 'latitude': 39.9428324, 'amenity': 'nan', 'shop': 'supermarket', 'opening_hours': 'Mo-Su 07:00-22:00', 'website': 'https://www.wholefoodsmarket.com/stores/southstreet', 'phone': '+1 215-733-9788'}


In [17]:
# Create a comprehensive map showing all waypoints
print("Creating comprehensive map of all waypoints...")

# Create map centered on Philadelphia
philly_map = folium.Map(
    location=[39.9526, -75.1652], 
    zoom_start=12,
    tiles='OpenStreetMap'
)

# Add bounding box rectangle to show search area
bbox_coords = [
    [39.9, -75.2],   # SW
    [40.0, -75.2],   # NW  
    [40.0, -75.1],   # NE
    [39.9, -75.1],   # SE
    [39.9, -75.2]    # Close rectangle
]
folium.PolyLine(bbox_coords, color='red', weight=3, opacity=0.8, popup='Search Area').add_to(philly_map)

# Add water fountains
water_count = 0
for wp in water_waypoints:
    lat, lon = wp['latitude'], wp['longitude']
    name = wp['name']
    amenity = wp['amenity']
    
    # Create popup text
    popup_text = f"""
    <b>Water Fountain</b><br>
    Name: {name}<br>
    Type: {amenity}<br>
    Coordinates: {lat:.6f}, {lon:.6f}
    """
    
    folium.Marker(
        [lat, lon],
        popup=folium.Popup(popup_text, max_width=200),
        icon=folium.Icon(color='blue', icon='tint', prefix='fa'),
        tooltip=f"Water: {name}"
    ).add_to(philly_map)
    water_count += 1

# Add stores
store_count = 0
for wp in store_waypoints:
    lat, lon = wp['latitude'], wp['longitude']
    name = wp['name']
    shop_type = wp['shop']
    
    # Create popup text
    popup_text = f"""
    <b>Store</b><br>
    Name: {name}<br>
    Type: {shop_type}<br>
    Coordinates: {lat:.6f}, {lon:.6f}
    """
    
    folium.Marker(
        [lat, lon],
        popup=folium.Popup(popup_text, max_width=200),
        icon=folium.Icon(color='green', icon='shopping-cart', prefix='fa'),
        tooltip=f"Store: {name}"
    ).add_to(philly_map)
    store_count += 1

# Add legend
legend_html = f'''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 200px; height: 120px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     font-size:14px; padding: 10px">
<p><b>Waypoints Found</b></p>
<p><i class="fa fa-tint" style="color:blue"></i> Water Fountains: {water_count}</p>
<p><i class="fa fa-shopping-cart" style="color:green"></i> Stores: {store_count}</p>
<p><b>Total: {water_count + store_count}</b></p>
</div>
'''
philly_map.get_root().html.add_child(folium.Element(legend_html))

print(f"Map created with {water_count} water fountains and {store_count} stores")
philly_map


Creating comprehensive map of all waypoints...
Map created with 9 water fountains and 198 stores


In [ ]:
# Save waypoint data to JSON files for API integration
waypoint_data = {
    'water_fountains': water_waypoints,
    'stores': store_waypoints,
    'metadata': {
        'total_water_fountains': len(water_waypoints),
        'total_stores': len(store_waypoints),
        'bbox': PHILLY_BBOX,
        'last_updated': pd.Timestamp.now().isoformat()
    }
}

# Save to JSON file
with open('waypoint_data.json', 'w', encoding='utf-8') as f:
    json.dump(waypoint_data, f, indent=2, ensure_ascii=False)

print(f"Saved waypoint data to waypoint_data.json")
print(f"Total water fountains: {len(water_waypoints)}")
print(f"Total stores: {len(store_waypoints)}")

# Also save as separate files for easier loading
with open('water_fountains.json', 'w', encoding='utf-8') as f:
    json.dump(water_waypoints, f, indent=2, ensure_ascii=False)

with open('stores.json', 'w', encoding='utf-8') as f:
    json.dump(store_waypoints, f, indent=2, ensure_ascii=False)

print("Also saved separate files: water_fountains.json and stores.json")

# Print summary
print(f"\n=== SUMMARY ===")
print(f"Search Area: Philadelphia ({PHILLY_BBOX})")
print(f"Water Fountains Found: {len(water_waypoints)}")
print(f"Stores Found: {len(store_waypoints)}")
print(f"Total Waypoints: {len(water_waypoints) + len(store_waypoints)}")

if water_waypoints:
    print(f"\nWater Fountain Locations:")
    for i, wp in enumerate(water_waypoints[:5]):  # Show first 5
        print(f"  {i+1}. {wp['name']} at {wp['coordinates']}")
    if len(water_waypoints) > 5:
        print(f"  ... and {len(water_waypoints) - 5} more")

if store_waypoints:
    print(f"\nStore Locations (first 5):")
    for i, wp in enumerate(store_waypoints[:5]):  # Show first 5
        print(f"  {i+1}. {wp['name']} ({wp['shop']}) at {wp['coordinates']}")
    if len(store_waypoints) > 5:
        print(f"  ... and {len(store_waypoints) - 5} more")


# Water Fountain & Store Data Fetcher

This notebook fetches water fountains and stores in Philadelphia to provide as additional context for pathfinding.

## Goals:
1. Fetch water fountain and store data from OSM
2. Create simple API endpoints to serve this data
3. Integrate with existing pathfinding without modifying the algorithm
4. Provide waypoint proximity information as additional context


In [1]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import json
from pyproj import Geod
from typing import List, Tuple, Dict, Any
import pandas as pd


/Users/romanus_angina/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## 1. Define Waypoint Tags and Fetch Data


In [2]:
# Define OSM tags for waypoints
WATER_TAGS = {
    'amenity': ['drinking_water']
}

STORE_TAGS = {
    'shop': ['convenience', 'supermarket', 'grocery', 'beverages']
}

# Philadelphia bounding box (same as in build_graph_2.py)
PHILLY_BBOX = (40.0, 39.9, -75.1, -75.2)  # (north, south, east, west)

print("Fetching water fountains and stores from OpenStreetMap...")


Fetching water fountains and stores from OpenStreetMap...


## 2. Fetch and Process Waypoint Data


In [5]:
# Fetch water fountains
print("Fetching water fountains...")
north, south, east, west = PHILLY_BBOX
# Use bbox parameter instead of individual north/south/east/west
water_gdf = ox.features_from_bbox(bbox=(west, south, east, north), tags=WATER_TAGS)
print(f"Found {len(water_gdf)} water fountains")

# Fetch stores
print("Fetching stores...")
store_gdf = ox.features_from_bbox(bbox=(west, south, east, north), tags=STORE_TAGS)
print(f"Found {len(store_gdf)} stores")

# Check what columns are available
print(f"\nWater fountains columns: {list(water_gdf.columns)}")
print(f"Stores columns: {list(store_gdf.columns)}")

# Display sample data (handle missing columns gracefully)
if len(water_gdf) > 0:
    print("\nSample water fountain:")
    available_cols = [col for col in ['amenity', 'name', 'geometry'] if col in water_gdf.columns]
    if available_cols:
        print(water_gdf[available_cols].head(1))
    else:
        print("No expected columns found")

if len(store_gdf) > 0:
    print("\nSample store:")
    available_cols = [col for col in ['shop', 'name', 'geometry'] if col in store_gdf.columns]
    if available_cols:
        print(store_gdf[available_cols].head(1))
    else:
        print("No expected columns found")


Fetching water fountains...
Found 9 water fountains
Fetching stores...
Found 325 stores

Water fountains columns: ['geometry', 'amenity', 'tourism', 'check_date', 'bottle', 'fountain', 'access', 'fee', 'wheelchair', 'dog']
Stores columns: ['geometry', 'addr:city', 'addr:housenumber', 'addr:postcode', 'addr:state', 'addr:street', 'branch', 'brand', 'brand:wikidata', 'check_date', 'check_date:opening_hours', 'name', 'opening_hours', 'phone', 'ref', 'shop', 'source', 'website', 'wheelchair', 'addr:housename', 'vacant', 'atm', 'delivery', 'fast_food', 'fuel:propane', 'cuisine', 'name:zh', 'wheelchair:description', 'contact:website', 'operator:wikidata', 'payment:app', 'payment:apple_pay', 'payment:cards', 'payment:cash', 'payment:contactless', 'payment:credit_cards', 'second_hand', 'opening_hours:signed', 'contact:phone', 'opening_hours:covid19', 'name:en', 'building', 'air_conditioning', 'changing_table', 'payment:debit_cards', 'toilets', 'drive_through', 'outdoor_seating', 'takeaway', 'i

## 3. Extract and Clean Waypoint Data


In [6]:
def extract_waypoint_data(gdf, waypoint_type: str) -> List[Dict[str, Any]]:
    """Extract waypoint data from GeoDataFrame into structured format"""
    waypoints = []
    for idx, row in gdf.iterrows():
        geom = row.geometry
        if geom and not geom.is_empty:
            if geom.geom_type == 'Point':
                lon, lat = geom.x, geom.y
                # Handle missing name column gracefully
                name = row.get('name', None) or row.get('amenity', None) or row.get('shop', None) or f'{waypoint_type}_{idx}'
                waypoint_data = {
                    'id': f"{waypoint_type}_{idx}",
                    'type': waypoint_type,
                    'name': str(name),
                    'coordinates': [lat, lon],  # [lat, lng] for frontend compatibility
                    'longitude': lon,
                    'latitude': lat,
                    'amenity': str(row.get('amenity', '')),
                    'shop': str(row.get('shop', '')),
                    'opening_hours': str(row.get('opening_hours', '')),
                    'website': str(row.get('website', '')),
                    'phone': str(row.get('phone', ''))
                }
                waypoints.append(waypoint_data)
            elif geom.geom_type == 'MultiPoint':
                for i, point in enumerate(geom.geoms):
                    lon, lat = point.x, point.y
                    name = row.get('name', None) or row.get('amenity', None) or row.get('shop', None) or f'{waypoint_type}_{idx}_{i}'
                    waypoint_data = {
                        'id': f"{waypoint_type}_{idx}_{i}",
                        'type': waypoint_type,
                        'name': str(name),
                        'coordinates': [lat, lon],
                        'longitude': lon,
                        'latitude': lat,
                        'amenity': str(row.get('amenity', '')),
                        'shop': str(row.get('shop', '')),
                        'opening_hours': str(row.get('opening_hours', '')),
                        'website': str(row.get('website', '')),
                        'phone': str(row.get('phone', ''))
                    }
                    waypoints.append(waypoint_data)
    return waypoints

# Extract waypoints
water_waypoints = extract_waypoint_data(water_gdf, 'water')
store_waypoints = extract_waypoint_data(store_gdf, 'store')

print(f"Extracted {len(water_waypoints)} water waypoints")
print(f"Extracted {len(store_waypoints)} store waypoints")

# Sample waypoints
if water_waypoints:
    print(f"\nSample water waypoint: {water_waypoints[0]}")
if store_waypoints:
    print(f"Sample store waypoint: {store_waypoints[0]}")


Extracted 9 water waypoints
Extracted 198 store waypoints

Sample water waypoint: {'id': "water_('node', 1586414753)", 'type': 'water', 'name': 'drinking_water', 'coordinates': [39.9662606, -75.1826961], 'longitude': -75.1826961, 'latitude': 39.9662606, 'amenity': 'drinking_water', 'shop': '', 'opening_hours': '', 'website': '', 'phone': ''}
Sample store waypoint: {'id': "store_('node', 333786275)", 'type': 'store', 'name': 'Whole Foods Market', 'coordinates': [39.9428324, -75.1581249], 'longitude': -75.1581249, 'latitude': 39.9428324, 'amenity': 'nan', 'shop': 'supermarket', 'opening_hours': 'Mo-Su 07:00-22:00', 'website': 'https://www.wholefoodsmarket.com/stores/southstreet', 'phone': '+1 215-733-9788'}


## 4. Create Proximity Functions for Waypoints


In [7]:
def find_nearby_waypoints(lat: float, lon: float, waypoints: List[Dict], max_distance: float = 500) -> List[Dict]:
    """Find waypoints within max_distance meters of given coordinates"""
    geod = Geod(ellps="WGS84")
    nearby = []
    
    for waypoint in waypoints:
        wp_lat, wp_lon = waypoint['latitude'], waypoint['longitude']
        distance = geod.inv(lon, lat, wp_lon, wp_lat)[2]  # geodesic distance in meters
        
        if distance <= max_distance:
            waypoint_with_distance = waypoint.copy()
            waypoint_with_distance['distance_meters'] = round(distance, 1)
            nearby.append(waypoint_with_distance)
    
    # Sort by distance
    nearby.sort(key=lambda x: x['distance_meters'])
    return nearby

def get_waypoints_along_path(path_coords: List[List[float]], waypoints: List[Dict], max_distance: float = 200) -> List[Dict]:
    """Find waypoints near any point along a path"""
    geod = Geod(ellps="WGS84")
    path_waypoints = []
    
    for waypoint in waypoints:
        wp_lat, wp_lon = waypoint['latitude'], waypoint['longitude']
        min_distance = float('inf')
        closest_path_point = None
        
        # Find closest point on path
        for i, (path_lat, path_lon) in enumerate(path_coords):
            distance = geod.inv(path_lon, path_lat, wp_lon, wp_lat)[2]
            if distance < min_distance:
                min_distance = distance
                closest_path_point = i
        
        if min_distance <= max_distance:
            waypoint_with_distance = waypoint.copy()
            waypoint_with_distance['distance_meters'] = round(min_distance, 1)
            waypoint_with_distance['path_index'] = closest_path_point
            path_waypoints.append(waypoint_with_distance)
    
    # Sort by path index
    path_waypoints.sort(key=lambda x: x['path_index'])
    return path_waypoints

# Test the proximity functions
test_lat, test_lon = 39.9526, -75.1652  # Center of Philadelphia
print(f"\nTesting proximity functions at {test_lat}, {test_lon}")

nearby_water = find_nearby_waypoints(test_lat, test_lon, water_waypoints, max_distance=1000)
nearby_stores = find_nearby_waypoints(test_lat, test_lon, store_waypoints, max_distance=1000)

print(f"Found {len(nearby_water)} water fountains within 1km")
print(f"Found {len(nearby_stores)} stores within 1km")

if nearby_water:
    print(f"Closest water fountain: {nearby_water[0]['name']} ({nearby_water[0]['distance_meters']}m)")
if nearby_stores:
    print(f"Closest store: {nearby_stores[0]['name']} ({nearby_stores[0]['distance_meters']}m)")



Testing proximity functions at 39.9526, -75.1652
Found 1 water fountains within 1km
Found 24 stores within 1km
Closest water fountain: drinking_water (40.4m)
Closest store: Wawa (177.2m)


## 5. Save Waypoint Data for API Integration


In [8]:
# Save waypoint data to JSON files for API integration
waypoint_data = {
    'water_fountains': water_waypoints,
    'stores': store_waypoints,
    'metadata': {
        'total_water_fountains': len(water_waypoints),
        'total_stores': len(store_waypoints),
        'bbox': PHILLY_BBOX,
        'last_updated': pd.Timestamp.now().isoformat()
    }
}

# Save to JSON file
with open('waypoint_data.json', 'w', encoding='utf-8') as f:
    json.dump(waypoint_data, f, indent=2, ensure_ascii=False)

print(f"Saved waypoint data to waypoint_data.json")
print(f"Total water fountains: {len(water_waypoints)}")
print(f"Total stores: {len(store_waypoints)}")

# Also save as separate files for easier loading
with open('water_fountains.json', 'w', encoding='utf-8') as f:
    json.dump(water_waypoints, f, indent=2, ensure_ascii=False)

with open('stores.json', 'w', encoding='utf-8') as f:
    json.dump(store_waypoints, f, indent=2, ensure_ascii=False)

print("Also saved separate files: water_fountains.json and stores.json")


Saved waypoint data to waypoint_data.json
Total water fountains: 9
Total stores: 198
Also saved separate files: water_fountains.json and stores.json


## 6. Test the Implementation


In [9]:
# Let's verify the coordinates are actually in Philadelphia
import folium

# Create a map centered on Philadelphia
philly_map = folium.Map(location=[39.9526, -75.1652], zoom_start=12)

# Add the waypoint we found
waypoint_lat, waypoint_lon = 39.9662606, -75.1826961
folium.Marker(
    [waypoint_lat, waypoint_lon], 
    popup=f"Water Fountain: {waypoint_lat}, {waypoint_lon}",
    icon=folium.Icon(color='blue', icon='tint')
).add_to(philly_map)

# Add a marker for Center City Philadelphia for reference
folium.Marker(
    [39.9526, -75.1652], 
    popup="Center City Philadelphia",
    icon=folium.Icon(color='red', icon='star')
).add_to(philly_map)

# Add bounding box rectangle
bbox_coords = [
    [39.9, -75.2],   # SW
    [40.0, -75.2],   # NW  
    [40.0, -75.1],   # NE
    [39.9, -75.1],   # SE
    [39.9, -75.2]    # Close rectangle
]
folium.PolyLine(bbox_coords, color='red', weight=2, opacity=0.7).add_to(philly_map)

philly_map


In [11]:
# Let's try a different approach to get more comprehensive data
print("Trying alternative OSM queries...")

# Try with more specific tags for water fountains
WATER_TAGS_ALT = {
    'amenity': ['drinking_water'],
}

# Try with more comprehensive store tags
STORE_TAGS_ALT = {
    'shop': ['*'],  # Get all shop types
    'amenity': ['marketplace', 'vending_machine']
}

print("Fetching with alternative water tags...")
water_gdf_alt = ox.features_from_bbox(bbox=(west, south, east, north), tags=WATER_TAGS_ALT)
print(f"Found {len(water_gdf_alt)} water features with alternative tags")

print("Fetching with alternative store tags...")
store_gdf_alt = ox.features_from_bbox(bbox=(west, south, east, north), tags=STORE_TAGS_ALT)
print(f"Found {len(store_gdf_alt)} store features with alternative tags")

# Check what we got
if len(water_gdf_alt) > 0:
    print(f"\nAlternative water features columns: {list(water_gdf_alt.columns)}")
    print("Sample alternative water feature:")
    print(water_gdf_alt.head(1))

if len(store_gdf_alt) > 0:
    print(f"\nAlternative store features columns: {list(store_gdf_alt.columns)}")
    print("Sample alternative store feature:")
    print(store_gdf_alt.head(1))


Trying alternative OSM queries...
Fetching with alternative water tags...
Found 9 water features with alternative tags
Fetching with alternative store tags...
Found 56 store features with alternative tags

Alternative water features columns: ['geometry', 'amenity', 'tourism', 'check_date', 'bottle', 'fountain', 'access', 'fee', 'wheelchair', 'dog']
Sample alternative water feature:
                                     geometry         amenity  tourism  \
element id                                                               
node    1586414753  POINT (-75.1827 39.96626)  drinking_water  artwork   

                   check_date bottle fountain access  fee wheelchair  dog  
element id                                                                 
node    1586414753        NaN    NaN      NaN    NaN  NaN        NaN  NaN  

Alternative store features columns: ['geometry', 'amenity', 'addr:city', 'addr:housenumber', 'addr:postcode', 'addr:street', 'check_date', 'name', 'opening_hours',

In [12]:
# Test with a sample path
sample_path = [
    [39.910075, -75.186375],  # Start
    [39.920000, -75.180000],  # Middle
    [39.938657, -75.173162]   # End
]

print("Testing waypoint detection along a sample path...")

# Find waypoints along the path
path_water = get_waypoints_along_path(sample_path, water_waypoints, max_distance=500)
path_stores = get_waypoints_along_path(sample_path, store_waypoints, max_distance=500)

print(f"Found {len(path_water)} water fountains along the path")
print(f"Found {len(path_stores)} stores along the path")

if path_water:
    print("\nWater fountains along path:")
    for wp in path_water[:3]:  # Show first 3
        print(f"  - {wp['name']} at {wp['coordinates']} ({wp['distance_meters']}m from path)")

if path_stores:
    print("\nStores along path:")
    for wp in path_stores[:3]:  # Show first 3
        print(f"  - {wp['name']} at {wp['coordinates']} ({wp['distance_meters']}m from path)")

print("\n✅ Waypoint data fetching and proximity functions are working!")
print("Next steps:")
print("1. Run this notebook to generate waypoint_data.json")
print("2. Copy the API integration code to app.py")
print("3. Update the frontend to display waypoints on the map")
print("4. Add waypoint information to pathfinding responses")


Testing waypoint detection along a sample path...
Found 0 water fountains along the path
Found 1 stores along the path

Stores along path:
  - ALDI at [39.9179634, -75.1842617] (428.8m from path)

✅ Waypoint data fetching and proximity functions are working!
Next steps:
1. Run this notebook to generate waypoint_data.json
2. Copy the API integration code to app.py
3. Update the frontend to display waypoints on the map
4. Add waypoint information to pathfinding responses
